In [1]:
# !pip install seqio
# !pip install t5

In [2]:
# !pip install -U huggingface_hub
# !pip install -U ipywidgets

In [1]:
import seqio
import functools
import torch
from datasets import Dataset
import t5.data
from t5.data import postprocessors
from t5.data import preprocessors
from t5.data.glue_utils import get_glue_metric
from t5.data.glue_utils import get_glue_postprocess_fn
from t5.data.glue_utils import get_glue_text_preprocessor
from t5.data.glue_utils import get_super_glue_metric
from t5.evaluation import metrics
import tensorflow_datasets as tfds
from t5.models import utils as model_utils
import gin
from absl import logging
from datasets import load_dataset,concatenate_datasets,DatasetDict
import tensorflow as tf

In [2]:
tfds.__version__

'4.5.2+nightly'

In [3]:
tfds.__version__

'4.5.2+nightly'

In [4]:
TaskRegistry = seqio.TaskRegistry



DEFAULT_OUTPUT_FEATURES = {
    "inputs": seqio.Feature(
        vocabulary=t5.data.get_default_vocabulary(), add_eos=True,
        required=False),
    "targets": seqio.Feature(
        vocabulary=t5.data.get_default_vocabulary(), add_eos=True)
}

# ==================================== C4 ======================================
# Final pretraining task used in Raffel et al., 2019.
TaskRegistry.add(
    "c4_v220_span_corruption",
    source=seqio.TfdsDataSource(tfds_name="c4/en:2.2.0"),
    preprocessors=[
        functools.partial(
            preprocessors.rekey, key_map={
                "inputs": None,
                "targets": "text"
            }),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        preprocessors.span_corruption,
        seqio.preprocessors.append_eos_after_trim,

    ],
    output_features=DEFAULT_OUTPUT_FEATURES,
    metric_fns=[])


# Baseline pretraining task used in Raffel et al., 2019.
TaskRegistry.add(
    "c4_v220_iid_denoising",
    source=seqio.TfdsDataSource(tfds_name="c4/en:2.2.0"),
    preprocessors=[
        functools.partial(
            preprocessors.rekey, key_map={
                "inputs": None,
                "targets": "text"
            }),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        preprocessors.iid_denoising,
        seqio.preprocessors.append_eos_after_trim,
    ],
    output_features=DEFAULT_OUTPUT_FEATURES,
    metric_fns=[])


# Prefix language modeling pretraining task used in Raffel et al., 2019.
TaskRegistry.add(
    "c4_v220_prefix_lm",
    source=seqio.TfdsDataSource(tfds_name="c4/en:2.2.0"),
    preprocessors=[
        functools.partial(
            preprocessors.rekey, key_map={
                "inputs": None,
                "targets": "text"
            }),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        preprocessors.prefix_lm,
        seqio.preprocessors.append_eos_after_trim,
    ],
    output_features=DEFAULT_OUTPUT_FEATURES,
    metric_fns=[])


# Configurable tasks used for comparisons in Raffel et al., 2019.
_c4_config_suffixes = ["", ".noclean", ".realnewslike", ".webtextlike"]
for config_suffix in _c4_config_suffixes:
    TaskRegistry.add(
        "c4{name}_v020_unsupervised".format(name=config_suffix.replace(".", "_")),
        source=seqio.TfdsDataSource(tfds_name="c4/en{config}:2.2.0".format(
          config=config_suffix)),
        preprocessors=[
          functools.partial(
              preprocessors.rekey, key_map={
                  "inputs": None,
                  "targets": "text"
              }),
          seqio.preprocessors.tokenize,
          seqio.CacheDatasetPlaceholder(),
          preprocessors.unsupervised,
          seqio.preprocessors.append_eos_after_trim,
        ],
        output_features=DEFAULT_OUTPUT_FEATURES,
        metric_fns=[])


# ================================ Wikipedia ===================================
TaskRegistry.add(
    "wikipedia_20190301.en_v003_unsupervised",
    source=seqio.TfdsDataSource(tfds_name="wikipedia/20190301.en:1.0.0"),
    preprocessors=[
        functools.partial(
            preprocessors.rekey, key_map={
                "inputs": None,
                "targets": "text"
            }),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        preprocessors.unsupervised,
        seqio.preprocessors.append_eos_after_trim,
    ],
    output_features=DEFAULT_OUTPUT_FEATURES,
    metric_fns=[])


# =================================== GLUE =====================================
for b in tfds.text.glue.Glue.builder_configs.values():
    TaskRegistry.add(
        "glue_%s_v002" % b.name,
        source=seqio.TfdsDataSource(
        tfds_name="glue/%s:2.0.0" % b.name,
        splits=["test"] if b.name == "ax" else None),
        preprocessors=[
        get_glue_text_preprocessor(b),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
        ],
        metric_fns=get_glue_metric(b.name),
        output_features=DEFAULT_OUTPUT_FEATURES,
        postprocess_fn=get_glue_postprocess_fn(b))

# =============================== CNN DailyMail ================================
TaskRegistry.add(
    "cnn_dailymail_v002",
    source=seqio.TfdsDataSource(tfds_name="cnn_dailymail:3.1.0"),
    preprocessors=[
        functools.partial(
            preprocessors.summarize,
            article_key="article",
            summary_key="highlights"),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    metric_fns=[metrics.rouge],
    output_features=DEFAULT_OUTPUT_FEATURES)

# ==================================== WMT =====================================
# Format: year, tfds builder config, tfds version
b_configs = [
    ("14", tfds.translate.wmt14.Wmt14Translate.builder_configs["de-en"], "1.0.0"
    ),
    ("14", tfds.translate.wmt14.Wmt14Translate.builder_configs["fr-en"], "1.0.0"
    ),
    ("16", tfds.translate.wmt16.Wmt16Translate.builder_configs["ro-en"], "1.0.0"
    ),
    ("15", tfds.translate.wmt15.Wmt15Translate.builder_configs["fr-en"], "1.0.0"
    ),
    ("19", tfds.translate.wmt19.Wmt19Translate.builder_configs["de-en"], "1.0.0"
    ),
]

for prefix, b, tfds_version in b_configs:
    TaskRegistry.add(
        "wmt%s_%s%s_v003" % (prefix, b.language_pair[1], b.language_pair[0]),
        source=seqio.TfdsDataSource(tfds_name="wmt%s_translate/%s:%s" %
                                  (prefix, b.name, tfds_version)),
        preprocessors=[
          functools.partial(
              preprocessors.translate,
              source_language=b.language_pair[1],
              target_language=b.language_pair[0],
          ),
          seqio.preprocessors.tokenize,
          seqio.CacheDatasetPlaceholder(),
          seqio.preprocessors.append_eos_after_trim,
        ],
        metric_fns=[metrics.bleu],
        output_features=DEFAULT_OUTPUT_FEATURES)

# Special case for t2t ende.
b = tfds.translate.wmt_t2t.WmtT2tTranslate.builder_configs["de-en"]
TaskRegistry.add(
    "wmt_t2t_ende_v003",
    source=seqio.TfdsDataSource(tfds_name="wmt_t2t_translate/de-en:1.0.0"),
    preprocessors=[
        functools.partial(
            preprocessors.translate,
            source_language=b.language_pair[1],
            target_language=b.language_pair[0]),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    metric_fns=[metrics.bleu],
    output_features=DEFAULT_OUTPUT_FEATURES)

# ================================= SuperGlue ==================================
for b in tfds.text.super_glue.SuperGlue.builder_configs.values():
  # We use a simplified version of WSC, defined below
    if "wsc" in b.name:
        continue
    if b.name == "axb":
        glue_preprocessors = [
            functools.partial(
                preprocessors.rekey,
                key_map={
                    "premise": "sentence1",
                    "hypothesis": "sentence2",
                    "label": "label",
                    "idx": "idx",
                }),
            get_glue_text_preprocessor(b),
            seqio.preprocessors.tokenize,
            seqio.CacheDatasetPlaceholder(),
            seqio.preprocessors.append_eos_after_trim,
        ]
    else:
        glue_preprocessors = [
            get_glue_text_preprocessor(b),
            seqio.preprocessors.tokenize,
            seqio.CacheDatasetPlaceholder(),
            seqio.preprocessors.append_eos_after_trim,
    ]
    TaskRegistry.add(
        "super_glue_%s_v102" % b.name,
        source=seqio.TfdsDataSource(
          tfds_name="super_glue/%s:1.0.2" % b.name,
          splits=["test"] if b.name in ["axb", "axg"] else None),
        preprocessors=glue_preprocessors,
        metric_fns=get_super_glue_metric(b.name),
        output_features=DEFAULT_OUTPUT_FEATURES,
        postprocess_fn=get_glue_postprocess_fn(b))

    # Create SuperGLUE tasks with 1 sentinel token added.
    seqio.experimental.add_task_with_sentinels("super_glue_%s_v102" % b.name,
                                             num_sentinels=1)

# ======================== Definite Pronoun Resolution =========================
TaskRegistry.add(
    "dpr_v001_simple",
    source=seqio.TfdsDataSource(tfds_name="definite_pronoun_resolution:1.1.0"),
    preprocessors=[
        preprocessors.definite_pronoun_resolution_simple,
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    metric_fns=[metrics.accuracy],
    output_features=DEFAULT_OUTPUT_FEATURES)

# Create SuperGLUE tasks with 1 sentinel token added.
seqio.experimental.add_task_with_sentinels("dpr_v001_simple", num_sentinels=1)

# =================================== WSC ======================================
TaskRegistry.add(
    "super_glue_wsc_v102_simple_train",
    source=seqio.TfdsDataSource(
        tfds_name="super_glue/wsc.fixed:1.0.2", splits=["train"]),
    preprocessors=[
        functools.partial(preprocessors.wsc_simple, correct_referent_only=True),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    metric_fns=[],
    output_features=DEFAULT_OUTPUT_FEATURES)

# Create SuperGLUE tasks with 1 sentinel token added.
seqio.experimental.add_task_with_sentinels("super_glue_wsc_v102_simple_train",
                                           num_sentinels=1)

TaskRegistry.add(
    "super_glue_wsc_v102_simple_eval",
    source=seqio.TfdsDataSource(
        tfds_name="super_glue/wsc.fixed:1.0.2", splits=["validation", "test"]),
    preprocessors=[
        functools.partial(
            preprocessors.wsc_simple, correct_referent_only=False),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    postprocess_fn=postprocessors.wsc_simple,
    metric_fns=[metrics.accuracy],
    output_features=DEFAULT_OUTPUT_FEATURES)
# Create SuperGLUE tasks with 1 sentinel token added.
seqio.experimental.add_task_with_sentinels("super_glue_wsc_v102_simple_eval",
                                           num_sentinels=1)

# =================================== WNLI =====================================
TaskRegistry.add(
    "glue_wnli_v002_simple_eval",
    source=seqio.TfdsDataSource(
        tfds_name="glue/wnli:1.0.0", splits=["validation", "test"]),
    preprocessors=[
        preprocessors.wnli_simple,
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    postprocess_fn=postprocessors.wsc_simple,
    metric_fns=[metrics.accuracy],
    output_features=DEFAULT_OUTPUT_FEATURES)

# =================================== Squad ====================================
# Maximized evaluation metrics over all answers.
TaskRegistry.add(
    "squad_v010_allanswers",
    source=seqio.TfdsDataSource(tfds_name="squad/v1.1:3.0.0"),
    preprocessors=[
        preprocessors.squad,
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    postprocess_fn=postprocessors.qa,
    metric_fns=[metrics.squad],
    output_features=DEFAULT_OUTPUT_FEATURES)


# Maximized evaluation metrics over all answers.
TaskRegistry.add(
    "squad_v010_context_free",
    source=seqio.TfdsDataSource(tfds_name="squad/v1.1:3.0.0"),
    preprocessors=[
        functools.partial(preprocessors.squad, include_context=False),
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    postprocess_fn=postprocessors.qa,
    metric_fns=[metrics.squad],
    output_features=DEFAULT_OUTPUT_FEATURES)

# Squad span prediction task instead of text.
TaskRegistry.add(
    "squad_v010_allanswers_span",
    source=seqio.TfdsDataSource(tfds_name="squad/v1.1:3.0.0"),
    preprocessors=[
        preprocessors.squad_span_space_tokenized,
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    postprocess_fn=postprocessors.span_qa,
    metric_fns=[metrics.span_squad],
    output_features=DEFAULT_OUTPUT_FEATURES)

# Deprecated: Use `squad_v010_allanswers` instead.
TaskRegistry.add(
    "squad_v010",
    source=seqio.TfdsDataSource(tfds_name="squad/v1.1:3.0.0"),
    preprocessors=[
        preprocessors.squad,
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        seqio.preprocessors.append_eos_after_trim,
    ],
    metric_fns=[metrics.squad],
    output_features=DEFAULT_OUTPUT_FEATURES)

# ================================= TriviaQA ===================================
TaskRegistry.add(
    "trivia_qa_v010",
    source=seqio.TfdsDataSource(tfds_name="trivia_qa/rc:1.1.0"),
    preprocessors=[
        preprocessors.trivia_qa,
        seqio.preprocessors.tokenize,
        seqio.CacheDatasetPlaceholder(),
        preprocessors.trivia_qa_truncate_inputs,
        seqio.preprocessors.append_eos_after_trim,
    ],
    metric_fns=[],
    output_features=DEFAULT_OUTPUT_FEATURES)


# =============== PrefixLM objectives (not used in the T5 paper) ===============


# # Vocabulary (shared by encoder and decoder)
# sentencepiece_model_file = "gs://t5-data/vocabs/cc_all.32000.100extra/sentencepiece.model"

# vocab = seqio.SentencePieceVocabulary(sentencepiece_model_file)

# seqio.TaskRegistry.add(
#     "c4_prefix_lm_objective_encoder_decoder_architecture",
#     source=seqio.TfdsDataSource(tfds_name="c4/en:2.2.0"),
#     preprocessors=[
#         functools.partial(
#             preprocessors.rekey, key_map={
#                 "inputs": None,
#                 "targets": "text"
#             }),
#         seqio.preprocessors.tokenize,
#         seqio.CacheDatasetPlaceholder(),
#         preprocessors.targets_for_prefix_lm_objective,
#         preprocessors.pack_prefix_lm_encoder_decoder,
#     ],
#     output_features={
#         "encoder_input_tokens": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "decoder_target_tokens": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "decoder_input_tokens": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "encoder_segment_ids": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "encoder_positions": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "decoder_segment_ids": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "decoder_positions": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "decoder_loss_weights": seqio.Feature(vocabulary=vocab, add_eos=False),
#         # All but the last stage of the preprocessing uses "targets" as the key,
#         # so this output feature is necessary. It is not marked required because
#         # the final preprocessor drops it.
#         "targets": seqio.Feature(vocabulary=vocab, required=False),
#     },
#     metric_fns=[])


# seqio.TaskRegistry.add(
#     "c4_prefix_lm_objective_decoder_architecture",
#     source=seqio.TfdsDataSource(tfds_name="c4/en:2.2.0"),
#     preprocessors=[
#         functools.partial(
#             preprocessors.rekey, key_map={
#                 "inputs": None,
#                 "targets": "text"
#             }),
#         seqio.preprocessors.tokenize,
#         seqio.CacheDatasetPlaceholder(),
#         preprocessors.targets_for_prefix_lm_objective,
#         preprocessors.pack_prefix_lm_decoder_only,
#     ],
#     output_features={
#         "decoder_target_tokens": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "decoder_input_tokens": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "decoder_loss_weights": seqio.Feature(vocabulary=vocab, add_eos=False),
#         "decoder_causal_attention": seqio.Feature(
#             vocabulary=vocab, add_eos=False),
#         # All but the last stage of the preprocessing uses "targets" as the key,
#         # so this output feature is necessary. It is not marked required because
#         # the final preprocessor drops it.
#         "targets": seqio.Feature(vocabulary=vocab, required=False),
#     },
#     metric_fns=[])


# TaskRegistry.add(
#     "c4_v220_full_lm",
#     source=seqio.TfdsDataSource(tfds_name="c4/en:2.2.0"),
#     preprocessors=[
#         functools.partial(
#             preprocessors.rekey, key_map={
#                 "inputs": None,
#                 "targets": "text"
#             }),
#         seqio.preprocessors.tokenize,
#         seqio.CacheDatasetPlaceholder(),
#         preprocessors.full_lm,
#     ],
#     output_features={
#         "targets": seqio.Feature(vocabulary=vocab, add_eos=True)
#     },
#     metric_fns=[])

# EDA

In [13]:
#glue_mnli_v002
#super_glue_cb_v102
dataset = seqio.get_mixture_or_task("squad_v010_allanswers").get_dataset(
    sequence_length={"inputs": 256, "targets": 128},
    split="train",
    shuffle=True,
    num_epochs=1,
    shard_info=seqio.ShardInfo(index=0, num_shards=10),
    use_cached=False,
    seed=42
)

iterator = dataset.as_numpy_iterator()

2022-03-27 22:20:42.824645: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [14]:
cnt=2
for i, ex in enumerate(iterator):
    print(ex.keys())
    print(ex)
    cnt-=1
    if cnt==0:
        break

dict_keys(['inputs_pretokenized', 'inputs', 'targets_pretokenized', 'targets', 'id', 'context', 'question', 'answers'])
{'inputs_pretokenized': b"question: What must be well understood when applying a servo motor for use ? context: A servomotor is a motor , very often sold as a complete module , which is used within a position - control or speed - control feedback control system mainly control valves , such as motor - operated control valves . Servomotors are used in applications such as machine tools , pen plotters , and other process systems . Motors intended for use in a servomechanism must have well - documented characteristics for speed , torque , and power . The speed vs . torque curve is quite important and is high ratio for a servo motor . Dynamic response characteristics such as winding inductance and rotor inertia are also important ; these factors limit the overall performance of the servomechanism loop . Large , powerful , but slow - responding servo loops may use conventio

# Create dataset



In [30]:
def get_date(name,split,dataset_name):
    
    start_position=len(name)+1
    
    if dataset_name=='superglue':
        seqio_name=f"super_glue_{name}_v102"
    elif dataset_name=='glue':
        seqio_name=f"glue_{name}_v002"
    elif dataset_name=='squad':
        seqio_name= 'squad_v010_allanswers'
    else :
        raise f"dataset_name: {dataset_name} not config"
    print(f'getting dataset == {seqio_name}')
    dataset = seqio.get_mixture_or_task(seqio_name).get_dataset(
        sequence_length={"inputs": 1, "targets": 1},
        split=split,
        shuffle=True,
        num_epochs=1,
        shard_info=seqio.ShardInfo(index=0, num_shards=1),
        use_cached=False,
        seed=42
    )

    iterator = dataset.as_numpy_iterator()

    dictionary = {
        "idx": [],
        "inputs": [],
        "targets": [],
    }
    
    if name == 'record':
        dictionary['answers']=[]

    for i, ex in enumerate(iterator):
        if name =='multirc':
            dictionary["idx"].append(f"{ex['idx/paragraph']}-{ex['idx/question']}-{ex['idx/answer']}") 
        elif name == 'record':
            dictionary["idx"].append(ex["idx/query"])
            dictionary['answers'].append([i.decode("utf-8") for i in ex["answers"]])
        elif name == 'squad':
            dictionary["idx"].append(ex["id"].decode("utf-8"))
            start_position=0
        else: dictionary["idx"].append(ex["idx"]) 

        dictionary["inputs"].append(ex["inputs_pretokenized"].decode("utf-8")[start_position:])
        dictionary["targets"].append(ex["targets_pretokenized"].decode("utf-8"))
    
    return Dataset.from_dict(dictionary)

## for mnli

In [68]:
seqio_mnli_dataset=DatasetDict()

In [69]:
name="mnli"
#super_glue_cb_v102
for split in ['train','test_matched','test_mismatched','validation_matched','validation_mismatched']:
    print(split)
    seqio_mnli_dataset[split]=get_date(name,split,'glue')

train
test_matched
test_mismatched
validation_matched
validation_mismatched


In [70]:
seqio_mnli_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 392702
    })
    test_matched: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 9847
    })
    validation_matched: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 9832
    })
})

In [71]:
# create validation and test dataset by combining matched and mismatched

seqio_mnli_dataset['validation']=concatenate_datasets([seqio_mnli_dataset['validation_matched'],seqio_mnli_dataset['validation_mismatched']])
seqio_mnli_dataset['test']=concatenate_datasets([seqio_mnli_dataset['test_matched'],seqio_mnli_dataset['test_mismatched']])

In [72]:
seqio_mnli_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 392702
    })
    test_matched: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 9847
    })
    validation_matched: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 9832
    })
    validation: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 19647
    })
    test: Dataset({
        features: ['idx', 'inputs', 'targets'],
        num_rows: 19643
    })
})

In [ ]:
seqio_mnli_dataset.push_to_hub(
        "stjokerli/TextToText_mnli_seqio",
)

## For superGlue

### For 'cb','boolq','rte','copa' ,'wic','multirc','record'

In [8]:
text_to_text_SuperGlue=DatasetDict()

In [13]:
# task_dict=
for name in [
#     'cb','boolq','rte','copa'
#             ,'wic',
#     'multirc',
    'record'
            ]:
   
    print(name)
    temp=DatasetDict()
    for split in ['train','test','validation']:
        
        print(" ",split)
        temp[split]=get_date(name,split,"superglue")
    text_to_text_SuperGlue[name]=temp

record
  train
  test
  validation


## For wcs

In [21]:
def get_data(name,subfix,split):
    
    start_position=len(name)+1
    
#     if dataset_name=='superglue':
#         seqio_name=f"super_glue_{name}_v102"
#     elif dataset_name=='glue':
#         seqio_name=f"glue_{name}_v002"
#     else :
#         raise f"dataset_name: {dataset_name} not config"
    seqio_name=f"super_glue_{name}_v102_simple_{subfix}"
    dataset = seqio.get_mixture_or_task(seqio_name).get_dataset(
        sequence_length={"inputs": 1, "targets": 1},
        split=split,
        shuffle=True,
        num_epochs=1,
        shard_info=seqio.ShardInfo(index=0, num_shards=1),
        use_cached=False,
        seed=42
    )

    iterator = dataset.as_numpy_iterator()

    dictionary = {
        "idx": [],
        "inputs": [],
        "targets": []
    }
    for i, ex in enumerate(iterator):
        dictionary["idx"].append(ex["idx"])
        dictionary["inputs"].append(ex["inputs_pretokenized"].decode("utf-8")[start_position:])
        dictionary["targets"].append(ex["targets_pretokenized"].decode("utf-8"))
    
    return Dataset.from_dict(dictionary)

In [22]:
temp=DatasetDict()
temp['train']=get_data('wsc','train','train')
temp['validation']=get_data('wsc','eval','validation')
temp['test']=get_data('wsc','eval','test')

In [23]:
text_to_text_SuperGlue['wsc']=temp

In [24]:
def get_date(name,split,dataset_name):
    
    start_position=len(name)+1
    
    if dataset_name=='superglue':
        seqio_name=f"super_glue_{name}_v102"
    elif dataset_name=='glue':
        seqio_name=f"glue_{name}_v002"
    else :
        raise f"dataset_name: {dataset_name} not config"
    dataset = seqio.get_mixture_or_task(seqio_name).get_dataset(
        sequence_length={"inputs": 1, "targets": 1},
        split=split,
        shuffle=True,
        num_epochs=1,
        shard_info=seqio.ShardInfo(index=0, num_shards=1),
        use_cached=False,
        seed=42
    )

    iterator = dataset.as_numpy_iterator()

    dictionary = {
        "idx": [],
        "inputs": [],
        "targets": []
    }
    for i, ex in enumerate(iterator):
        dictionary["idx"].append(ex["idx"])
        dictionary["inputs"].append(ex["inputs_pretokenized"].decode("utf-8")[start_position:])
#         print(ex["targets_pretokenized"])
        dictionary["targets"].append(ex["targets_pretokenized"].decode("utf-8"))
    
    return Dataset.from_dict(dictionary)

In [25]:
for name in ['axg','axb']:
    
    print(name)
    temp=DatasetDict()
    for split in ['test']:
        
        print(" ",split)
        temp[split]=get_date(name,split,"superglue")
    text_to_text_SuperGlue[name]=temp

axg
  test
axb
  test


In [26]:
text_to_text_SuperGlue

DatasetDict({
    cb: DatasetDict({
        train: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 250
        })
        test: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 250
        })
        validation: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 56
        })
    })
    boolq: DatasetDict({
        train: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 9427
        })
        test: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 3245
        })
        validation: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 3270
        })
    })
    rte: DatasetDict({
        train: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 2490
        })
        test: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_r

# push to hugging face data hub

In [18]:
for task in text_to_text_SuperGlue.keys():
    text_to_text_SuperGlue[task].push_to_hub(
        f"stjokerli/TextToText_{task}_seqio",
)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
text_to_text_SuperGlue

DatasetDict({
    record: DatasetDict({
        train: Dataset({
            features: ['idx', 'inputs', 'targets', 'answers'],
            num_rows: 138854
        })
        test: Dataset({
            features: ['idx', 'inputs', 'targets', 'answers'],
            num_rows: 10000
        })
        validation: Dataset({
            features: ['idx', 'inputs', 'targets', 'answers'],
            num_rows: 15176
        })
    })
})

# add squad

In [16]:
text_to_text_SuperGlue=DatasetDict()

In [31]:
# task_dict=
for name in [
#     'cb','boolq','rte','copa'
#             ,'wic',
#     'multirc',
#     'record'
    'squad'
            ]:
   
    print(name)
    temp=DatasetDict()
    for split in ['train','validation']:
        
        print(" ",split)
        temp[split]=get_date(name,split,"squad")
    text_to_text_SuperGlue[name]=temp

squad
  train
getting dataset == squad_v010_allanswers
  validation
getting dataset == squad_v010_allanswers


In [32]:
text_to_text_SuperGlue

DatasetDict({
    squad: DatasetDict({
        train: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 87599
        })
        validation: Dataset({
            features: ['idx', 'inputs', 'targets'],
            num_rows: 10570
        })
    })
})

In [33]:
text_to_text_SuperGlue['squad']['train'][0]

{'idx': '573632816c16ec1900b92992',
 'inputs': 'question: What does trophy hunting create in Africa ? context: A scientific study in the journal , Biological Conservation , states that trophy hunting is of " major importance to conservation in Africa by creating economic incentives for conservation over vast areas , including areas which may be unsuitable for alternative wildlife - based land uses such as photographic ecotourism . " However , another study states that less than 3 % of a trophy hunters \' expenditures reach the local level , meaning that the economic incentive and benefit is " minimal , particularly when we consider the vast areas of land that hunting concessions occupy . " ',
 'targets': 'economic incentives for conservation'}

In [35]:
for task in text_to_text_SuperGlue.keys():
    text_to_text_SuperGlue[task].push_to_hub(
        f"stjokerli/TextToText_{task}_seqio",
)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]